In [1]:
!pip install -U langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 KB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 KB 7.5 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.13
    Uninstalling langsmith-0.1.13:
      Successfully uninstalled langsmith-0.1.13
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.28
    Uninstalling langchain-core-0.1.28:
      Successfully uninstalled langchain-core-0.1.28
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.25
    Uninstalling langchain-community-0.0.25:
      Successfully uninstalled langchain-community-0.0.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.10
    Uninstalli

In [2]:
!pip install langchain-openai

# LLM Chain

In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
llm.invoke("how can langsmith help with testing?")

'\nLangSmith is an AI-powered language model that can assist with testing in several ways:\n\n1. Automated Testing: LangSmith can be used to generate automated tests for software applications. By using natural language processing and machine learning algorithms, LangSmith can analyze the code and identify potential bugs or errors, allowing developers to focus on writing test cases that cover the most critical areas of the application.\n2. Code Review: LangSmith can assist in code review by highlighting potential issues in the codebase, such as security vulnerabilities, performance bottlenecks, or coding standards violations. Developers can then review these findings and make necessary adjustments to improve the quality of the code.\n3. Test Data Generation: LangSmith can generate test data automatically, which can save developers time and effort in creating test cases. By using natural language processing algorithms, LangSmith can understand the requirements of the application and gene

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is an incredible tool that can greatly assist with testing. Here are some ways in which Langsmith can help:\n\n1. Automated Testing: With Langsmith, you can automate your testing process by generating test cases based on your documentation. This saves time and effort compared to manually writing test cases, ensuring that all possible scenarios are covered.\n2. Consistency: Langsmith's AI-powered testing tool helps maintain consistency in your test cases, ensuring that the tests are comprehensive and accurate. This reduces the risk of overlooking critical test cases or including unnecessary ones.\n3. Improved Test Coverage: By analyzing your documentation, Langsmith can identify potential gaps in test coverage and suggest additional test cases to ensure comprehensive testing. This leads to a more robust testing process and fewer chances of missing critical issues.\n4. Faster Testing Cycles: With Langsmith's au

In [6]:
from langchain_core.output_parsers import StrOutputParser

In [7]:
output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LangSmith is an excellent tool for testing and validating content. Here are some ways in which LangSmith can help with testing:\n\n1. Language Analysis: LangSmith's language analysis capabilities can help identify potential issues with the content, such as grammar, spelling, and punctuation errors. By analyzing the text through LangSmith, you can identify areas that require improvement, ensuring that your technical documentation is accurate and error-free.\n2. Consistency Checking: LangSmith can help ensure consistency in your technical documentation by checking for inconsistencies in terminology, formatting, and styling. This is particularly useful when working on large documents or projects with multiple authors, as it helps maintain a consistent tone and style throughout the content.\n3. Clarity Evaluation: LangSmith's clarity evaluation feature can help assess the readability of your technical documentation. By ana

# Retrieval Chain

In [10]:
!pip install beautifulsoup4

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [13]:
!pip install faiss-cpu

In [14]:
!pip install langchain_text_splitters

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context: 

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain) 

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


According to the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. Debugging: When developing new LLM applications, LangSmith tracing is enabled by default, making it easier to debug where the model is failing. The platform provides clear visibility and debugging information at each step of an LLM sequence.
3. Initial test set: LangSmith allows developers to create datasets for initial test cases, which can be uploaded in bulk, created on the fly, or exported from application traces.
4. Comparison view: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.
5. Beta testing: LangSmith supports beta testing, which allows developers to collect more data on how their LLM applications are 

# Conversation Retrieval Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder 

# Frist we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"), 
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

# A new chain takes in the most recent input and the coversation history and use an LLM to generate a search query
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
# Test by using a predefined chat histroy
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
# Create a new chain to continue the conversation with these retrieved documents in mind
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's quesitions based on the below context:\n\n{context}"), 
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
# End-to-end test

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

# Agent

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access the Tavily API key
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
# The retriever tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever, 
    "langsmith_search", 
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!", 
)

In [ ]:
# The Tavily search tool
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [ ]:
tools = [retriever_tool, search]
#tools = [search]

In [ ]:
!pip install langchainhub

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})